In [1]:
from pathlib import Path
from typing import List, Dict

from Models.LocalizedString import LocalizedString, ParamTypes, LocalizedStringParam
import Services.GoogleSheetsService as sheetService
from Services.LocalFilesService import LocalFilesService, isWindows

## Constants and configurations

In [2]:
filesBaseDir = Path("./CachedFiles")
filePathLocalizedStringRaw = filesBaseDir / "LocalizedStringRaw.json"
filePathLocalizedStringGroups = filesBaseDir / "LocalizedStringGroups.json"

fileService = LocalFilesService(baseFolder=filesBaseDir)

In [3]:
defaultLanguage = 'RU'
codeFileTab = '    '

# filePathUniqueMessage = Path('./bot-content.unique-message.ts')
filePathUniqueMessage = Path('../src/core/bot-content/schemas/Models/bot-content.unique-message.ts')

In [4]:
cachedTrueValue = 'TRUE'

languagesStartSheetLetter = 'F'
languagesStartSheetLetterIndex = ord(languagesStartSheetLetter)

firstLetter = 'A'
firstLetterIndex = ord(firstLetter)

lastLetter = 'Z'
lastLetterIndex = ord(lastLetter)

# Starts from 1
configurationRowIndex = 5
startSheetLocalizationRowsIndex = 6
endSheetLocalizationRowsIndex = 1000

## Cache languages configuration

In [5]:
languages = sheetService.getContent(
    page=sheetService.pages.uniqueMessage,
    range=f'{languagesStartSheetLetter}{configurationRowIndex}:{lastLetter}{configurationRowIndex}'
)[0]

languagesCount = len(languages)
rowEstimatedLength = languagesStartSheetLetterIndex - firstLetterIndex + languagesCount
print(languages)
print(f'Languages count: {languagesCount}')
print(f'Estimated row length: {rowEstimatedLength}')

['RU', 'EN', 'AR', 'TR', 'UK', 'BE', 'KK']
Languages count: 7
Estimated row length: 12


In [6]:
def makeSpreadSheetPlaceholderWithIndex(index: int) -> str:
    return f'<#{index}>'

def makeIosPlaceholderWithIndex(index: int) -> str:
    return f'%{index}$@'

## Cache spreadsheet localization content

In [7]:
sheetStringsContent = sheetService.getContent(
    page=sheetService.pages.uniqueMessage, 
    range=f'{firstLetter}{startSheetLocalizationRowsIndex}:{chr(languagesStartSheetLetterIndex + languagesCount)}{endSheetLocalizationRowsIndex}'
)
sheetStringsContent = [row for row in sheetStringsContent if len(row) >= rowEstimatedLength]
len(sheetStringsContent)

32

In [8]:
def parseParams(rawParamsString: str) -> List[LocalizedStringParam]:
    result: List[LocalizedStringParam] = []
    for row in rawParamsString.split('\n'):
        rowComponents = [ component.strip() for component in row.split('/') ]
        if len(rowComponents) == 3 and rowComponents[2] in ParamTypes._member_names_:
            result.append(LocalizedStringParam(
                paramKey=rowComponents[0],
                name=rowComponents[1],
                paramType=ParamTypes[rowComponents[2]]
            ))
    return result

In [9]:
localizedStrings: List[LocalizedString] = []
for row in sheetStringsContent:
    localizedValues = row[rowEstimatedLength - languagesCount:]
    localizedValuesDict: Dict[str, str] = {}
    for index, localizedValue in enumerate(localizedValues):
        localizedValuesDict[languages[index]] = localizedValue

    localizedStrings.append(
        LocalizedString(
            group=row[0],
            key=row[1],
            comment=row[2],
            parameters=parseParams(row[3]),
            isUniqueMessage=row[4] == cachedTrueValue,
            localizedValues=localizedValuesDict
        )
    )
print(localizedStrings)


[LocalizedString(group='Sample', key='sample', comment='Тестовый текст', parameters=[], isUniqueMessage=False, localizedValues={'RU': 'Пример', 'EN': 'Example', 'AR': 'مثال', 'TR': 'Örnek', 'UK': 'Приклад', 'BE': 'Прыклад', 'KK': 'Мысал'}), LocalizedString(group='Common', key='unknownState', comment='', parameters=[], isUniqueMessage=True, localizedValues={'RU': 'Я тебя не понимаю 😔\n⬇️ Нажми на кнопку снизу ⬇️', 'EN': "I don't understand you 😔\n⬇️ Click on the below button ⬇️", 'AR': 'أنا لا أفهمك 😔\n⬇ انقر على الزر أدناه ⬇', 'TR': 'Seni anlamıyorum 😔\n⬇️ Aşağıdaki düğmeye tıklayın ⬇️', 'UK': 'Я вас не розумію 😔\n⬇ натисніть кнопку нижче ⬇', 'BE': 'Я цябе не разумею 😔\n⬇ Націсніце на кнопку ніжэй ⬇', 'KK': 'Мен сені түсінбеймін 😔\n⬇️ Төмендегі батырманы басыңыз ⬇️'}), LocalizedString(group='Common', key='permissionDenied', comment='', parameters=[], isUniqueMessage=True, localizedValues={'RU': 'У вас нет доступа к этому разделу', 'EN': 'You have no access to this section', 'AR': 'ليس 

## Unique messages generation

In [10]:
def classNameFrom(text: str) -> str:
    return text[0].upper() + text[1:]

def propertyNameFrom(text: str) -> str:
    return text[0].lower() + text[1:]

In [11]:
localizedGroups: Dict[str, List[LocalizedString]] = {}
for localizedString in localizedStrings:

    if localizedString.isUniqueMessage is False: continue
    
    if localizedString.group not in localizedGroups:
        localizedGroups[localizedString.group] = []
    localizedGroups[localizedString.group].append(localizedString)

In [12]:
def generateClassCodeForGroup(groupName: str, localizedValues: List[LocalizedString]) -> str:
    
    result = f'export class {classNameFrom(groupName)} '
    result += '{'

    for localizedString in localizedValues:
        
        comment = codeFileTab + '/**'
        comment += f'\n{codeFileTab} * {localizedString.localizedValues[defaultLanguage]}'
        comment += f'\n{codeFileTab} */'
        # result += f'\n{comment}'
        
        localizedValueSample = localizedString.localizedValues[defaultLanguage].replace('\n', '\\n')
        result += f'\n{codeFileTab}readonly {propertyNameFrom(localizedString.key)} = \'{localizedValueSample}\''
    
    result += '\n}'
    return result

In [13]:
uniqueMessagesFileContent = '''// ==================
// * Generated file *
// ==================

export class UniqueMessage {'''

for group in localizedGroups:
    uniqueMessagesFileContent += f'\n{codeFileTab}readonly {propertyNameFrom(group)} = new {classNameFrom(group)}()'

uniqueMessagesFileContent += '\n}\n'

for group, localizedString in localizedGroups.items():
    uniqueMessagesFileContent += f'\n{generateClassCodeForGroup(group, localizedString)}\n'

# print(uniqueMessagesFileContent)

In [14]:
if isWindows:
    with filePathUniqueMessage.open('w', encoding='utf-8') as file:
        file.write(uniqueMessagesFileContent)
else:
    with filePathUniqueMessage.open('w') as file:
        file.write(uniqueMessagesFileContent)